# Introducción a la librería spaCy
En este *notebook* vamos a describir el uso de la librería `spaCy` para el Procesado de Lenguaje Natural.

In [ ]:
import spacy
import pandas as pd
import re

Cargamos la librería y el modelo de lenguaje para el español. Vemos las principales características de la librería y del modelo.

In [ ]:
spacy.info()

Comprobamos los modelos de lenguaje instalados (compatible con todas las versiones de `spaCy`)

In [ ]:
!python -m spacy validate

In [ ]:
nlp = spacy.load("es_core_news_md")

In [ ]:
spacy.info('es_core_news_md')

### Información del modelo
Podemos ver todas las características del modelo cargado con su atributo `meta` (diccionario)

In [ ]:
nlp.meta.keys()

In [ ]:
nlp.meta['pipeline']

## Arquitectura del modelo
Para cada modelo, `spaCy` tiene un vocabulario de palabras conocidas (*lexemes*) que almacena en un `stringStore` global

Cada modelo tiene un conjunto de lexemas del idioma definidos en su Vocabulario

In [ ]:
nlp.vocab

In [ ]:
print("\nTamaño vocabulario: ", len(nlp.vocab)) #esta longitud es falsa

In [ ]:
len(nlp.vocab.strings)

`spaCy` no precargar en memoria todo el vocabulario, sino que carga los lexemas conforme los va necesitando:  
> *To reduce the initial loading time, the lexemes in `nlp.vocab` are no longer loaded on initialization for models with vectors. As you process texts, the lexemes will be added to the vocab automatically, just as in small models without vectors.*

In [ ]:
for w in nlp.vocab.vectors:
    _ = nlp.vocab[w]

print("\nTamaño vocabulario: ", len(nlp.vocab))

In [ ]:
nlp.vocab["ciudad"]

In [ ]:
nlp.vocab["ciudad"].text

In [ ]:
[prop for prop in dir(spacy.lexeme.Lexeme) if not prop.startswith('_')]

In [ ]:
nlp.vocab["adiós"].has_vector

In [ ]:
"jhgsfhgjsf" in nlp.vocab

In [ ]:
nlp.vocab["jhgsfhgjsf"]

In [ ]:
"jhgsfhgjsf" in nlp.vocab

In [ ]:
nlp.vocab["adiós"].orth

In [ ]:
nlp.vocab.strings[15970597814306712899]

## Procesado de texto
spaCy ejecuta todos los análisis del texto con una sola instrucción. Esta instrucción ejecuta un *pipeline* (procesado secuencial) que implementa:  

- División en tokens  
- Lematizado
- Análisis gramatical
- Análisis de dependencias
- *Name Entity Recognition* (NER)

In [ ]:
# Ejemplo de texto
texto = "La casa de Juan es muy bonita."

Lo primero que hacemos es procesar el texto y generar un objeto de tipo `Doc`

In [ ]:
doc = nlp(texto)
type(doc)

In [ ]:
[prop for prop in dir(spacy.tokens.doc.Doc) if not prop.startswith('_')]

In [ ]:
doc.text

In [ ]:
type(doc.text)

In [ ]:
doc

## Exploramos el documento

Al analizar un texto, spaCy lo divide en una lista de `tokens`, que se acceden iterando sobre el objeto `Doc`


In [ ]:
[t for t in doc]

In [ ]:
len(doc)

In [ ]:
doc[-1]

Esta división es distinta de la simple división por espacios:

In [ ]:
texto.split(' ')

In [ ]:
type(doc[0])

Cada token tiene una serie de atributos:

In [ ]:
print([prop for prop in dir(spacy.tokens.token.Token) if not prop.startswith('_')])

Cada propiedad tiene dos atributos:  
- `propiedad`: ID único o *hash* que identifica el valor en un diccionario común a todos los Docs (`stringstore`)
- `propiedad_`: valor de la propiedad  

Veamos los atributos de algunos de los tokens:

In [ ]:
token=doc[3]
print("TOKEN:", token)
print("original:", token.orth, token.orth_)
print("lowercased:", token.lower, token.lower_)
print("lemma:", token.lemma, token.lemma_)
print("shape:", token.shape, token.shape_)
print("prefix:", token.prefix, token.prefix_)
print("suffix:", token.suffix, token.suffix_)
print("log probability:", token.prob)
print("Brown cluster id:", token.cluster)
print("POS:", token.pos, token.pos_)
print("tag:", token.tag, token.tag_)
print("morphology:", token.morph)
print("Dependency parsing:", token.dep, token.dep_)


Las propiedades más importantes son (https://spacy.io/api/token#attributes):  
* `orth_`: texto del token
* `lemma_`: lema (palabra base)
* `shape_`: forma ortográfica del token
* `pos_`: Part-of-Speech (genérico)
* `tag_`: POS detallado
* `morph`: Análisis morfológico
* `dep_`: Tipo de dependencia del token (análisis de dependencias)

In [ ]:
pd.set_option('display.max_colwidth', None)

datos = map(lambda t: {'token': t.orth_,
                       'lema': t.lemma_,
                       'shape': t.shape_,
                       'POS': t.pos_,
                       'Descripción TAG': spacy.explain(t.tag_),
                       'dependencia': t.dep_,
                       'Descripción dep': spacy.explain(t.dep_),
                       'Morfología': t.morph}, doc)

pd.DataFrame(datos)

### Diferencia entre token y lexema

In [ ]:
nlp_en = spacy.load('en_core_web_md')
parsedData = nlp_en("I run a long run")
datos = map(lambda t: {'token': t.orth_,
                       'lema': t.lemma_,
                       'shape': t.shape_,
                       'POS': t.pos_,
                       'Morfología': t.morph,
                       'dependencia': t.dep_,
                       'Descripción dep': spacy.explain(t.dep_)},
                        parsedData)

pd.DataFrame(datos)

In [ ]:
parsedData[1]

In [ ]:
parsedData[4]

In [ ]:
parsedData[1]==parsedData[4]

In [ ]:
parsedData[1].orth

In [ ]:
nlp.vocab.strings[parsedData[1].orth]

In [ ]:
parsedData[1].orth==parsedData[4].orth

In [ ]:
parsedData[1].pos_

In [ ]:
parsedData[4].pos_

## Análisis gramatical
Los documentos SpaCy también dividen en texto en oraciones (*sentences*) que son objetos del tipo `spacy.tokens.span.Span`. Podemos iterar con el generador `doc.sents` usando `next()`, `list()`, un bucle o con una comprensión de lista.

In [ ]:
texto = "Al Sr. Daniel siempre le gustaron las catedrales. La de su ciudad era tan alta, \
que al mirarla desde su pequeña estatura, tenía que torcer el cuello de tal forma que \
le costaba no marearse. Lo que más temía era que sus pies se despegasen de la tierra \
y la Catedral le arrastrara con ella hasta los cielos. Aun así, un espíritu \
aventurero le llevaba cada tarde hasta la Plaza Mayor."
doc = nlp(texto)

In [ ]:
doc.text

In [ ]:
frases = doc.sents
frases

In [ ]:
next(frases)

In [ ]:
type(next(doc.sents))

In [ ]:
for i, sent in enumerate(doc.sents):
    print("Oración {}:\n{}\n".format(i,sent))

Cada sentencia también tiene sus propios atributos, distintos de los de los tokens. Para spaCy las oraciones son objetos de tipo `spacy.tokens.span.Span`

In [ ]:
print([prop for prop in dir(spacy.tokens.span.Span) if not prop.startswith('_')])

Algunas propiedades de `Span` son distintas que las de cada `Token`:

In [ ]:
[prop for prop in dir(spacy.tokens.span.Span) if
 not prop.startswith('_') and not prop in dir(spacy.tokens.token.Token)]

In [ ]:
frase=next(doc.sents)
frase

In [ ]:
frase.start

In [ ]:
frase.end

In [ ]:
[t for t in doc[frase.start:frase.end]]

In [ ]:
frase.root

In [ ]:
type(frase.root)

### Ejercicio 1
Obtén la palabra raíz -atributo `root`- de cada oración del texto anterior y muéstrala.  
La respuesta es  
```python  
[gustaron, alta, era, llevaba]
```

### Part of Speech (POS)
La librería `spaCy` determina el tipo gramatical (POS) de cada palabra en nuestro texto. Creamos un diccionario con los distintos POS de nuestro texto de ejemplo, usando el *hash* de cada POS como clave del diccionario:

In [ ]:
{w.pos: (w.pos_, spacy.explain(w.pos_)) for w in doc} 

Cada tipo gramatical (POS) se subdivide en distintas etiquetas según su análisis mnorfológico(atributo `morph`).  
Por ejemplo en nuestro texto tenemos los siguientes `tag`:

In [ ]:
pd.DataFrame(set([(w.pos_, w.morph) for w in doc]), columns=['POS', 'morph']).sort_values(by='POS')

### Ejercicio 2
Crea una lista de Python con todas las palabras del texto `doc` que sean del tipo NOMBRE y además sean de género Femenino.  
Ayuda: tendrás que usar una comprensión de lista filtrando mediante una función de búsqueda de texto (o patrón regular) sobre el valor del atributo `morph` 

Podemos extraer el género de todos los sustantivos a partir de su atributo `morph`:

In [ ]:
import re

[(w, re.findall('Gender=(\w+)', str(w.morph))) for w in doc if w.pos_=='NOUN']

### Análisis de dependencias (dependency parsing)
La librería `spaCy` también analiza las relaciones entre palabras de una frase.

In [ ]:
doc = nlp("El perro de Juan se comió mi bocadillo.")
dependencias = [(t.text, t.dep_, spacy.explain(t.dep_)) for t in doc]
pd.DataFrame(list(dependencias), columns=['texto', 'dependencia', 'explicación'])

Cada palabra tiene un tipo de dependencia determinado dentro de la frase. 
La lista completa está en https://spacy.io/docs/api/annotation  
La dependencia `root` coincide con el atributo `root` de cada sentencia.

In [ ]:
sent=next(doc.sents)
sent.root

Cada raíz tiene una serie de hijos (tokens que dependen gramaticalmente de esa raíz).

In [ ]:
list(sent.root.children)

In [ ]:
doc[1].head

In [ ]:
doc[1].dep_

Podemos extender el análisis a cada palabra del texto.

In [ ]:
for word in sent: 
    print(word, ': ', str(list(word.children)))

Además, cada palabra tiene una palabra de la que depende (`.head`) y unas palabras que dependen de ella (`children`) a izquierda (`.lefts`) y a derecha (`.rights`).

In [ ]:
dependencias = map(lambda t: {
    'Palabra': t.orth_,
    'tipo de dependencia': f"{t.dep_} ({spacy.explain(t.dep_)})",
    'HEAD': t.head.orth_},
    doc)

pd.DataFrame(dependencias)

In [ ]:
dependencias = map(lambda token: {
    'dep. izquierdas': [t.orth_ for t in token.lefts],
    'palabra[tipo de dependencia]': f"{token.orth_} [{token.dep_}]",
    'dep.derechas': [t.orth_ for t in token.rights]},
    doc)

pd.DataFrame(dependencias)

Podemos representar gráficamente las dependencias con el módulo de visualización `displaCy`:

In [ ]:
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True, options={'distance':120})

También se pueden obtener los **sintagmas nominales** (*noun phrases*) de la oración. Cada NP tiene un sustantivo como raíz, acompañado ocasionalmente de las palabras que describen el sustantivo.  

In [ ]:
chunks = map(lambda chunk: {'NP': chunk.text,
                            'root': chunk.root.text,
                            'Dep.': chunk.root.dep_,
                            'head': chunk.root.head.text},
             doc.noun_chunks)

pd.DataFrame(chunks)

# Búsqueda de patrones de token
Spacy tiene una clase `Matcher` que permite buscar tokens con un patrón definido en los objetos `Doc`.  
Se puede buscar por el texto del token o por los atributos del token.  
Ref: https://spacy.io/usage/rule-based-matching

In [ ]:
from spacy.matcher import Matcher

#inicializamos sobre el vocabulario
matcher = Matcher(nlp.vocab)

In [ ]:
#definimos un patrón de texto a buscar
patron = [{"TEXT": "iPhone"}, {"TEXT": "X"}] #Patrón: texto 'iPhone' seguido de texto 'X'
matcher.add("iphone_x", [patron])

#procesamos un documento con el patrón
doc = nlp("El iPhone X salió después del iPhone 8, pero nunca sacaron el iPhone 9")

#llamamos al matcher
matches = matcher(doc)

#iteramos sobre los resultados
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

In [ ]:
matches #lista de 'matches'

In [ ]:
nlp.vocab.strings[1738708750870670527]

In [ ]:
doc[1:3]

Podemos buscar por atributos del token:

In [ ]:
patron = [{"TEXT": "iPhone"}, {"IS_DIGIT": True}] #Patrón: texto 'iPhone' seguido token con la atributo 'IS_DIGIT' a True
matcher.add("iphone_NN", [patron])
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre los resultados
for match_id, start, end in matches:
    matched_span = doc[start:end] #span del match en el documento
    string_id = nlp.vocab.strings[match_id] #identificador del match
    print(f"{string_id}: {matched_span.text}")

In [ ]:
doc = nlp("A mí me gusta el baile y a Pedro le gustaba tocar la trompeta pero no le gusta María y el gusta el iPhone X")

patron = [{"LEMMA": "gustar"}, {"POS": "DET", "OP": "?"}, {"POS": {"REGEX": "NOUN|PROPN"}}]
matcher.add("gustar_nombre", [patron])
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre el resultado
for match_id, start, end in matches:
    matched_span = doc[start:end]
    string_id = nlp.vocab.strings[match_id]
    print(f"{string_id}: {matched_span.text}")

### Ejercicio 3
Crea un nuevo patrón para el lema "gustar" seguido de un verbo

### Ejercicio 4
Busca todas las secuencias de texto formadas por nombre seguido de al menos un adjetivo en el texto siguiente.

In [ ]:
texto = "En el agua muerta, de una brillantez de estaño, permanecía inmóvil la barca-correo: \
un gran ataúd cargado de personas y paquetes, con la borda casi a flor de agua. La vela triangular, \
con remiendos obscuros, estaba rematada por un guiñapo incoloro que en otros tiempos había sido una \
bandera española destartalada y delataba el carácter oficial de la vieja embarcación."


# búsqueda de patrones de doc
También existe una clase para buscar secuencias de tokens (objetos tipo `doc`) dentro de un texto procesado. Se usa la clase `PhraseMatcher`, como explica https://spacy.io/usage/rule-based-matching#phrasematcher.  
Es útil para buscar un listado de palabras o frases en un texto.  

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
paises = ["Alemania", "Países Bajos", "Estados Unidos"]
#definimos patrones como objetos 'doc'
patterns = [nlp.make_doc(text) for text in paises] #sólo tokenizamos
matcher.add("Países", patterns)

doc = nlp("El presidente de Estados Unidos, Barack Obama, "
          "visitó al caciller de Alemania en Berlín")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)


In [ ]:
matcher = PhraseMatcher(nlp.vocab, attr="LEMMA")
colores = ["azul", "amarillo", "rojo", "verde"]
#definimos patrones como objetos 'doc'
patterns = [nlp(text) for text in colores] #hay que ejecutar todo el pipeline
matcher.add("Colores", patterns)

doc = nlp("Me gusta el azul pero los amarillos me parecen todos feos")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)